# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
schools = "Resources/schools_complete.csv"
students = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
schools_df = pd.read_csv(schools)
students_df = pd.read_csv(students)

In [2]:
# Combine the data into a single dataset
df = pd.merge(students_df, schools_df, how="left", on="school_name")
df.head()

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
tot_schools = schools_df['School ID'].count()
tot_students = students_df['Student ID'].count()
tot_budget = schools_df['budget'].sum()
avg_math = students_df['math_score'].mean()
avg_reading = students_df['reading_score'].mean()
passing_math = (students_df['Student ID'].loc[students_df['math_score']>=70].count()/tot_students)*100
passing_reading = (students_df['Student ID'].loc[students_df['reading_score']>=70].count()/tot_students)*100
avg_overall = (avg_math+avg_reading)/2

In [4]:
student_summ_df = pd.DataFrame({
    "Total Schools":[tot_schools],
    "Total Students":"{:,}".format(tot_students),
    "Total Budget":"${:,.2f}".format(tot_budget),
    "Avg Math Score":"{:.2f}".format(avg_math),
    "Avg Reading Score":"{:.2f}".format(avg_reading),
    "Avg Overall Score":"{:.2f}".format((avg_math+avg_reading)/2),
    "% Passing Math":"{:.2f}".format(passing_math),
    "% Passing Reading":"{:.2f}".format(passing_reading)
    })
student_summ_df

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
student_budget = schools_df['budget']/schools_df['size']
school_group = df.groupby(['school_name'])
avg_math_school = pd.DataFrame({
    "avg_math":school_group['math_score'].mean()
    })
avg_reading_school = pd.DataFrame({
    "avg_reading":school_group['reading_score'].mean()
    })
passing_math_school = pd.DataFrame({
    "passing_math":(df.loc[df['math_score'] >= 70, :].groupby('school_name')['math_score'].count()/school_group['size'].max())*100
    })
passing_reading_school = pd.DataFrame({
    "passing_reading":(df.loc[df['reading_score'] >= 70, :].groupby('school_name')['reading_score'].count()/school_group['size'].max())*100
    })
passing_overall_school = pd.DataFrame({
    "passing_overall":(passing_reading_school['passing_reading'] + passing_math_school['passing_math'])/2
    })

In [6]:
schools_df['student_budget'] = student_budget
schools_df = pd.merge(schools_df, avg_math_school, on="school_name",how="left")
schools_df = pd.merge(schools_df, avg_reading_school, on="school_name",how="left")
schools_df = pd.merge(schools_df, passing_math_school, on="school_name",how="left")
schools_df = pd.merge(schools_df, passing_reading_school, on="school_name",how="left")
schools_df = pd.merge(schools_df, passing_overall_school, on="school_name",how="left")
schools_df.head()

In [11]:
school_summ_df = pd.DataFrame({"School Name":schools_df['school_name'],
                        "School Type":schools_df['type'],
                        "Total Students":schools_df['size'].apply("{:,}".format),
                        "Total Budget":schools_df['budget'].apply("${:,.2f}".format),
                        "Per Student Budget":schools_df['student_budget'].apply("${:,.2f}".format),
                        "Avg Math Score":schools_df['avg_math'].apply("{:.3f}".format),
                        "Avg Reading Score":schools_df['avg_reading'].apply("{:.3f}".format),
                        "% Passing Math":schools_df['passing_math'].apply("{:.3f}".format),
                        "% Passing Reading":schools_df['passing_reading'].apply("{:.3f}".format),
                        "% Passing Overall":schools_df['passing_overall'].apply("{:.3f}".format)
                        })
school_summ_df = school_summ_df.set_index("School Name")
school_summ_df.index.name = None
school_summ_df

,School Type,Total Students,Total Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,73.500
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,73.364
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359,83.726,93.867,95.855,94.861
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,73.808
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.392,97.139,95.266
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274,83.989,93.868,96.540,95.204
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,95.587
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,74.307
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,94.379
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,95.270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [12]:
best_overall = school_summ_df.sort_values('% Passing Overall',ascending=False)
school_summ_df.index.name = None
best_overall.head()

,School Type,Total Students,Total Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,95.587
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418,83.849,93.272,97.309,95.291
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,95.270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.392,97.139,95.266
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274,83.989,93.868,96.540,95.204


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [13]:
worst_overall = school_summ_df.sort_values('% Passing Overall')
school_summ_df.index.name = None
worst_overall.head()

,School Type,Total Students,Total Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.843,80.745,66.367,80.220,73.293
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,73.364
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,73.500
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,73.640
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,73.804


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
ninth = students_df[students_df['grade'] == '9th']
ninth_math = pd.DataFrame({
    "9th":ninth.groupby('school_name')['math_score'].mean()
    })
tenth = students_df[students_df['grade'] == '10th']
tenth_math = pd.DataFrame({
    "10th":tenth.groupby('school_name')['math_score'].mean()
    })
eleventh = students_df[students_df['grade'] == '11th']
eleventh_math = pd.DataFrame({
    "11th":eleventh.groupby('school_name')['math_score'].mean()
    })
twelfth = students_df[students_df['grade'] == '12th']
twelfth_math = pd.DataFrame({
    "12th":twelfth.groupby('school_name')['math_score'].mean()
    })

In [17]:
math_summ_df = pd.DataFrame({
    "9th":ninth_math["9th"].apply("{:.2f}".format),
    "10th":tenth_math["10th"].apply("{:.2f}".format),
    "11th":eleventh_math["11th"].apply("{:.2f}".format),
    "12th":twelfth_math["12th"].apply("{:.2f}".format)
    })
math_summ_df = math_summ_df.rename_axis("School Name")
math_summ_df.index.name = None
math_summ_df

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [19]:
ninth_reading = pd.DataFrame({
    "9th":ninth.groupby('school_name')['reading_score'].mean()
    })
tenth_reading = pd.DataFrame({
    "10th":tenth.groupby('school_name')['reading_score'].mean()
    })
eleventh_reading = pd.DataFrame({
    "11th":eleventh.groupby('school_name')['reading_score'].mean()
    })
twelfth_reading = pd.DataFrame({
    "12th":twelfth.groupby('school_name')['reading_score'].mean()
    })

In [20]:
reading_summ_df = pd.DataFrame({
    "9th":ninth_reading["9th"].apply("{:.2f}".format),
    "10th":tenth_reading["10th"].apply("{:.2f}".format),
    "11th":eleventh_reading["11th"].apply("{:.2f}".format),
    "12th":twelfth_reading["12th"].apply("{:.2f}".format)
    })
reading_summ_df = reading_summ_df.rename_axis("School Name")
reading_summ_df.index.name = None
reading_summ_df

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [21]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [22]:
schools_df["Spending Per Student"] = pd.cut(schools_df["student_budget"], spending_bins, labels=group_names)
spending_df = schools_df.groupby("Spending Per Student")["avg_math","avg_reading","passing_math","passing_reading","passing_overall"].mean()
spending_df = spending_df.rename(columns={"avg_math":"Avg Math Score","avg_reading":"Avg Reading Score","passing_math":"% Passing Math","passing_reading":"% Passing Reading","passing_overall":"% Passing Overall"})
spending_df = spending_df.style.format("{:.2f}")
spending_df.index.name = None
spending_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
<$585,83.46,83.93,93.46,96.61,95.04
$585-615,83.60,83.89,94.23,95.90,95.07
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


## Scores by School Size

* Perform the same operations as above, based on school size.

In [23]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [24]:
schools_df["School Size"] = pd.cut(schools_df["size"], size_bins, labels=group_names)
size_df = schools_df.groupby("School Size")["avg_math","avg_reading","passing_math","passing_reading","passing_overall"].mean()
size_df = size_df.rename(columns={"avg_math":"Avg Math Score","avg_reading":"Avg Reading Score","passing_math":"% Passing Math","passing_reading":"% Passing Reading","passing_overall":"% Passing Overall"})
size_df = size_df.style.format("{:.2f}")
size_df.index.name = None
size_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


## Scores by School Type

* Perform the same operations as above, based on school type.

In [25]:
type_df = schools_df.groupby("type")["avg_math","avg_reading","passing_math","passing_reading","passing_overall"].mean()
type_df = type_df.rename(columns={"avg_math":"Avg Math Score","avg_reading":"Avg Reading Score","passing_math":"% Passing Math","passing_reading":"% Passing Reading","passing_overall":"% Passing Overall"})
type_df = type_df.rename_axis("School Type")
type_df = type_df.style.format("{:.2f}")
type_df.index.name = None
type_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
